In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("titanic/train.csv")

In [4]:
def fillnan(df):
    df["Age"] = df["Age"].fillna(method='ffill')
    df["Fare"] = df["Fare"].fillna(method='ffill')
    print(df.isnull().sum())

In [5]:
fillnan(df)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
X=df.iloc[:,[2,4,5,9]]

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     891 non-null    float64
 3   Fare    891 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 28.0+ KB


In [8]:
def maskX(X):
    X['Sex'].mask(X['Sex'] == 'female', 0, inplace=True)
    X['Sex'].mask(X['Sex'] == 'male', 1, inplace=True)

In [9]:
maskX(X)


C:\Users\saravanan\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


In [10]:
X["Age"].mean()

29.581560044893376

In [11]:
#X["Age"]=2.71**-X["Age"]
#print(X["Age"])

In [12]:
y=df.iloc[:,1]

In [13]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [16]:
X_train

,Pclass,Sex,Age,Fare
520,1,0,30.0,93.5000
372,3,1,19.0,8.0500
425,3,1,18.0,7.2500
31,1,0,40.0,146.5208
373,1,1,22.0,135.6333
...,...,...,...,...
498,1,0,25.0,151.5500
728,2,1,25.0,26.0000
712,1,1,48.0,52.0000
675,3,1,18.0,7.7750


In [17]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
model=LogisticRegression().fit(X_train,y_train)

In [19]:
a1=model.predict(X_test)
len(a1)


295

In [20]:
model.score(X_test, y_test)

0.7762711864406779

In [21]:
sum(y_test==a1)

229

In [22]:
#SVM
from sklearn import svm
model2=svm.SVC(kernel='linear').fit(X_train,y_train)


In [23]:
a2=model2.predict(X_test)

In [24]:
sum(y_test==a2)

234

In [25]:
model2.score(X_test,y_test)

0.7932203389830509

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
model3=DecisionTreeClassifier(max_depth=None,random_state=101,max_features=None,min_samples_leaf=15)
model3.fit(X_train,y_train)

DecisionTreeClassifier(min_samples_leaf=15, random_state=101)

In [28]:
a3=model3.predict(X_test)
model3.score(X_test,y_test)

0.847457627118644

In [29]:
from sklearn.naive_bayes import GaussianNB

In [30]:
model4 = GaussianNB().fit(X_train, y_train)
y_pred_gnb =model4.predict(X_test)
a4=model4.predict(X_test)
model4.score(X_test,y_test)

0.7762711864406779

In [31]:
from sklearn.ensemble import VotingClassifier

In [32]:
voting_classifier_hard = VotingClassifier(
    estimators = [('model3',DecisionTreeClassifier(random_state=101)),
                  ('model1', LogisticRegression()),
                  ('model4', GaussianNB()),
                 ('model2',svm.SVC(kernel='linear'))], )

In [33]:
voting_classifier_hard.fit(X_train, y_train)
y_pred_vch = voting_classifier_hard.predict(X_test)

In [34]:
voting_classifier_hard.score(X_test,y_test)

0.7966101694915254

In [36]:
df1=pd.read_csv("titanic/test.csv")

In [37]:
X_req=df1.iloc[:,[1,3,4,8]]

In [38]:
maskX(X_req)
fillnan(X_req)

Pclass    0
Sex       0
Age       0
Fare      0
dtype: int64


C:\Users\saravanan\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(
<ipython-input-4-dab49f76e7ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Age"] = df["Age"].fillna(method='ffill')
<ipython-input-4-dab49f76e7ad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [39]:
X_ans=df1["PassengerId"]

In [40]:
result=voting_classifier_hard.predict(X_req)

In [41]:
X_ans["Survived"]=result

<ipython-input-41-888f420733ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ans["Survived"]=result


In [42]:
X_ans

0                                                         892
1                                                         893
2                                                         894
3                                                         895
4                                                         896
                                  ...                        
414                                                      1306
415                                                      1307
416                                                      1308
417                                                      1309
Survived    [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ...
Name: PassengerId, Length: 419, dtype: object

In [43]:
output = pd.DataFrame({'PassengerId': df1.PassengerId, 'Survived': result})
output.to_csv('submission.csv', index=False)